In [20]:
from portfolio_optimization_class import PortfolioOptimization, optimize_windows
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
number_of_quarters = 32
tau = 0.05
maximum_weight = 0.3
data_path = r'data\random_data\n_stocks_per_sector.csv'
portfolios = optimize_windows(number_of_quarters, tau, maximum_weight, data_path)

In [4]:
copulas = ["clayton_random", "gaussian", "t_student"]
distributions = ["gauss_dist", "t_dist"]

In [ ]:
for copula in copulas:
    for window in range(0, 64):
        for dist in distributions:
            simulated_returns_file_path = r'../data_return_rates/simulated_' + copula + '_RETURNS_22_stocks' + f'_{window}_window_{dist}.csv'
            returns = pd.read_csv(simulated_returns_file_path)


In [11]:
# portfolios

In [12]:
returns_df = pd.read_csv(r'../data_return_rates/simulated_clayton_random_RETURNS_22_stocks_0_window_gauss_dist.csv', index_col=0)
# returns_df

In [ ]:
weights = portfolios.iloc[0].filter(regex="^w\d+$").values
expectile = -portfolios.iloc[0]["EVAR"]


weighted_returns = returns_df.values * weights


validation_values = (1 - tau) * np.minimum(weighted_returns - expectile, 0) - tau * np.maximum(weighted_returns - expectile, 0)
scoring_values = (1 - tau) * np.minimum((weighted_returns - expectile)**2, 0) - tau * np.maximum((weighted_returns - expectile)**2, 0)


expected_validation_values = np.mean(validation_values, axis=1)
expected_scoring_values = np.mean(scoring_values, axis=1)


validation_df = pd.DataFrame({
    'Portfolio Return': np.sum(weighted_returns, axis=1),
    'Expected Validation Value': expected_validation_values,
    'Expected Scoring Value': expected_scoring_values
})


In [40]:
expectile

0.021702123687977807

In [41]:
np.minimum(weighted_returns - expectile, 0)

array([[-0.026591518396296126, -0.021702123687977807,
        -0.01778623743457599, ..., 0, -0.021702123687977807,
        -0.021702123687977807],
       [-0.04216324990942255, -0.021702123687977807,
        -0.022273725774527527, ..., 0, -0.021702123687977807,
        -0.021702123687977807],
       [-0.011787908382052868, -0.021702123687977807,
        -0.015571420580864088, ..., -0.00202416277277713,
        -0.021702123687977807, -0.021702123687977807],
       ...,
       [-0.022184198833261327, -0.021702123687977807,
        -0.0042201685676119295, ..., 0, -0.021702123687977807,
        -0.021702123687977807],
       [-0.016431081955448377, -0.021702123687977807,
        -0.031394234669563634, ..., -0.06369631306153055,
        -0.021702123687977807, -0.021702123687977807],
       [-0.028802832364026438, -0.021702123687977807,
        -0.012562768998284322, ..., -0.023069173680573155,
        -0.021702123687977807, -0.021702123687977807]], dtype=object)

In [33]:
validation_df

,Portfolio Return,Expected Validation Value,Expected Scoring Value
0,0.10086,-0.0176772,-2.20157e-05
1,0.114682,-0.0192511,-3.84328e-05
2,0.0806146,-0.0173248,-1.86963e-05
3,0.114554,-0.0199823,-4.43027e-05
4,-0.082605,-0.0241841,-3.60575e-05
...,...,...,...
99995,0.0926844,-0.0186417,-2.4631e-05
99996,-0.0424479,-0.0224504,-3.17224e-05
99997,0.201915,-0.016306,-2.60242e-05
99998,-0.0724143,-0.0244707,-4.18344e-05


In [37]:
len(returns_df.values[0])

22

In [39]:
len(weighted_returns)

100000